In [13]:
import random # library for random number generation
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes

import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

from sklearn.cluster import KMeans 
#from sklearn.datasets.samples_generator import make_blobs

!pip install bs4
from bs4 import BeautifulSoup
import requests

from geopy.geocoders import Nominatim 
import requests
! conda install -c conda-forge folium
#import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
from collections import OrderedDict
import matplotlib as mpl

from pandas.io.json import json_normalize
import folium

import json

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [9]:
#Foursquare Info
CLIENT_ID = 'ABC' # your Foursquare ID
CLIENT_SECRET = 'ABC' # your Foursquare Secret
VERSION = '20210615'
Limit = 300

#other search parameters
city = 'Zürich'


#funtion to search for a query near a city, returns json file
def SearchVenues(query):
    
    search_results=[]
    for searchtopic in zip(query):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&query={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION,
            city, 
            Limit,
            query)
        results = requests.get(url).json()['response']['groups'][0]['items']
    
        search_results.append([(
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng']) for v in results])
    
        return (search_results)

In [10]:
#creating json file with wanted content
query1 = 'restaurant'
restaurants = SearchVenues(query1)
restaurants

KeyError: 'groups'

In [11]:
query2 = 'drinks'
bars = SearchVenues(query2)
query3 = 'train station'
train = SearchVenues(query3)
query4 = 'tram station'
tram = SearchVenues(query5)
query5 = 'bus station'
bus = SearchVenues(query5)

KeyError: 'groups'

In [12]:
df_rest = pd.DataFrame(item for venue_list in restaurants for item in venue_list)
df_bars = pd.DataFrame(item for venue_list in bars for item in venue_list)
df_trai = pd.DataFrame(item for venue_list in train for item in venue_list)
df_tram = pd.DataFrame(item for venue_list in tram for item in venue_list)
df_bus = pd.DataFrame(item for venue_list in bus for item in venue_list)
df_bars.head(5)

NameError: name 'bars' is not defined

In [ ]:
#combining all public tranpsort location to single dataframe
df_transport = df_trai.append(df_tram)
df_transport = df_transport.append(df_bus)

#edit column titles
df_rest.columns = ('Name', 'Lat', 'Lng')
df_bars.columns = ('Name', 'Lat', 'Lng')
df_transport.column = ('Name', 'Lat', 'Lng')
dt_transport.reset_index

df_rest.head(5)

In [ ]:
#view shape
print(df_rest.shape)
print(df_bars.shape)
print(df_transport.shape)

In [ ]:
#setting up k-means
def kmeanslabels(df):
    k_means = KMeans(init="k-means++", n_clusters=12, n_init=30)
    k_means.fit(df[['Lat','Lng']])
    k_means_labels = k_means.labels_
    return k_means_labels

def kmeanscenters(df):
    k_means = KMeans(init="k-means++", n_clusters=12, n_init=30)
    k_means.fit(df[['Lat','Lng']])
    k_means_cluster_centers = k_means.cluster_centers_
    return k_meansR_cluster_centers

In [ ]:
#Create k-means data for all three data sets
RestL = kmeanslabels(df_rest)
RC = kmeanscenters(df_rest)
BarL = kmeanslabels(df_bars)
BC = kmeanscenters(df_bars)
TransportL = kmeanslabels(df_transport)
TC = kmeanscenters(df_transport)

In [ ]:
#append kmeans cluster number to dafa frame for easy colouring
df_rest['Cluster Nr.'] = RestL
df_bars['Cluster Nr.'] = BarL
df_transport['Cluster Nr.'] = TransportL

In [ ]:
#create data frames for density centers
RestC = pd.DataFrame(RC)
BarC = pd.DataFrame(BC)
TransC = pd.DataFrame(TC)

In [ ]:
#plotting map
address = 'Zuerich'
geolocator = Nominatim(user_agent="zuerich_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

#setting cluster number as 6
k=12


def CreateMapK(df):
    # create map
    MyMap = folium.Map(location=[latitude, longitude], zoom_start=11)
    
    # setting color scheme for the clusters (R)
    X = np.arange(k)
    Ys = [i + X + (i*X)**2 for i in range(k)]
    colors_array = cm.rainbow(np.linspace(0, 1, len(Ys)))
    rainbow = [colors.rgb2hex(i) for i in colors_array]

    # adding markers to the map (R)
    markers_colors = []
    for lat, lng, counter in zip(df['Lat'], df['Lng'], df['Cluster Nr.']):
        label = folium.Popup(str(df['Cluster Nr.']) + ' Cluster ' + str(counter), parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=rainbow[counter-1],
            fill=True,
            fill_color=rainbow[counter-1],
            fill_opacity=0.7).add_to(MyMap)
    
    return MyMap

In [ ]:
#Creating map for seperate clusters
MapRest = CreateMapK(df_rest)
MapBar = CreateMapK(df_bars)
MapTrans = CreateMapK(df_transport)

In [ ]:
MapRest

In [ ]:
MapBar

In [ ]:
MapTrans

In [ ]:
#simple maps of density centers
def CreateMap(df):
    # create map
    MyMap = folium.Map(location=[latitude, longitude], zoom_start=11)
    
    # adding markers to the map (R)
    markers_colors = []
    for lat, lng, counter in zip(df['Lat'], df['Lng']):
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=rainbow,
            fill=True,
            fill_color=rainbow,
            fill_opacity=0.7).add_to(MyMap)
    
    return MyMap

In [ ]:
MapRestC = CreateMap(df_rest)
MapBarC = CreateMap(df_bars)
MapTransC = CreateMap(df_transport)